# Tutorial Goal

This tutorial aims to show how to **configure and run** a **synthetic workload** using the **wlgen module** provided by LISA.

# Configure logging

In [ ]:
import logging
from conf import LisaLogging
LisaLogging.setup()

In [1]:
# Execute this cell to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Other python modules required by this notebook
import json
import os

# Test environment setup

In [3]:
# Setup a target configuration
conf = {
    
    # Target is localhost
    "platform"    : 'linux',
    "board"       : "juno",
    
    # Login credentials
    "host"        : "192.168.0.1",
    "username"    : "root",
    "password"    : "",

    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # Comment the following line to force rt-app calibration on your target
#     "rtapp-calib" : {
#        "0": 355, "1": 138, "2": 138, "3": 355, "4": 354, "5": 354
#     },
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
             "sched_switch",
             "cpu_frequency"
         ],
         "buffsize" : 10240
    },

    # Where results are collected
    "results_dir" : "WlgenExample",
    
    # Devlib modules we'll need
    "modules": ["cpufreq"]
}

In [4]:
# Support to access the remote target
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(conf)
target = te.target

12:22:12  INFO    :         Target - Using base path: /home/derkling/Code/lisa
12:22:12  INFO    :         Target - Loading custom (inline) target configuration
12:22:12  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
12:22:12  INFO    :         Target - Connecting linux target:
12:22:12  INFO    :         Target -   username : root
12:22:12  INFO    :         Target -       host : 192.168.0.1
12:22:12  INFO    :         Target -   password : 
12:22:50  INFO    :         Target - Initializing target workdir:
12:22:50  INFO    :         Target -    /root/devlib-target
12:22:59  INFO    :         Target - Topology:
12:22:59  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
12:23:01  INFO    :       Platform - Loading default EM:
12:23:01  INFO    :       Platform -    /home/derkling/Code/lisa/libs/utils/platforms/juno.json
12:23:02  INFO    :         FTrace - Enabled tracepoints:
12:23:02  INFO    :         FTrace -   sched_switch
12:23:02  INFO    :   

## Workload execution utility

In [27]:
def execute(te, wload, res_dir):
    
    logging.info('# Create results folder for this execution')
    !mkdir {res_dir}
    
    logging.info('# Setup FTrace')
    te.ftrace.start()

    logging.info('## Start energy sampling')
    te.emeter.reset()

    logging.info('### Start RTApp execution')
    wload.run(out_dir=res_dir)

    logging.info('## Read energy consumption: %s/energy.json', res_dir)
    nrg_report = te.emeter.report(out_dir=res_dir)

    logging.info('# Stop FTrace')
    te.ftrace.stop()

    trace_file = os.path.join(res_dir, 'trace.dat')
    logging.info('# Save FTrace: %s', trace_file)
    te.ftrace.get_trace(trace_file)

    logging.info('# Save platform description: %s/platform.json', res_dir)
    plt, plt_file = te.platform_dump(res_dir)
    
    logging.info('# Report collected data:')
    logging.info('   %s', res_dir)
    !ls -la {res_dir}
    
    return nrg_report, plt, plt_file, trace_file

# Single task RTApp workload

## 1) creation

In [28]:
# Support to configure and run RTApp based workloads
from wlgen import RTA

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp_name = 'example1'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

12:38:37  INFO    :          WlGen - Setup new workload example1


## 2) configuration

In [29]:
# RTApp configurator for generation of PERIODIC tasks
from wlgen import Periodic 

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 100, # period [ms]
            duty_cycle_pct =  20, # duty cycle [%]
            duration_s     =  5,  # duration [s]
            delay_s        =  0,  # start after that delay [s]
            sched          = {    # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 130,
            },
            cpus           =      # pinned on first online CPU
                              str(target.list_online_cpus()[0])
            # ADD OTHER PARAMETERS
        ).get(),

    },
    
);

12:38:39  INFO    :          RTApp - Workload duration defined by longest task
12:38:39  INFO    :          RTApp - Default policy: SCHED_OTHER
12:38:39  INFO    :          RTApp - ------------------------
12:38:39  INFO    :          RTApp - task [task_p20], sched: {'policy': 'OTHER', 'priotity': 130}
12:38:39  INFO    :          RTApp -  | calibration CPU: 1
12:38:39  INFO    :          RTApp -  | loops count: 1
12:38:39  INFO    :          RTApp -  | CPUs affinity: 0
12:38:39  INFO    :          RTApp - + phase_000001: duration 5.000000 [s] (50 loops)
12:38:39  INFO    :          RTApp - |  period   100000 [us], duty_cycle  20 %
12:38:39  INFO    :          RTApp - |  run_time  20000 [us], sleep_time  80000 [us]


In [30]:
# Inspect the JSON file used to run the application
with open('./{}_00.json'.format(rtapp_name), 'r') as fh:
    rtapp_json = json.load(fh)
logging.info('Generated RTApp JSON file:')
print json.dumps(rtapp_json, indent=4, sort_keys=True)

12:38:42  INFO    : Generated RTApp JSON file:


{
    "global": {
        "calibration": 138, 
        "default_policy": "SCHED_OTHER", 
        "duration": -1, 
        "logdir": "/root/devlib-target"
    }, 
    "tasks": {
        "task_p20": {
            "cpus": [
                0
            ], 
            "loop": 1, 
            "phases": {
                "p000001": {
                    "loop": 50, 
                    "run": 20000, 
                    "timer": {
                        "period": 100000, 
                        "ref": "task_p20"
                    }
                }
            }, 
            "policy": "SCHED_OTHER", 
            "priotity": 130
        }
    }
}


## 3) execution

In [31]:
res_dir = os.path.join(te.res_dir, rtapp_name)
nrg_report, plt, plt_file, trace_file = execute(te, rtapp, res_dir)

12:38:45  INFO    : # Create results folder for this execution
12:38:45  INFO    : # Setup FTrace
12:38:49  INFO    : ## Start energy sampling
12:38:51  INFO    : ### Start RTApp execution
12:38:51  INFO    :          WlGen - Workload execution START:
12:38:51  INFO    :          WlGen -    /root/devlib-target/bin/rt-app /root/devlib-target/example1_00.json
12:38:57  INFO    : ## Read energy consumption: /home/derkling/Code/lisa/results/WlgenExample/example1/energy.json
12:38:57  INFO    :   EnergyReport - Energy [             a53]: 5.125284
12:38:57  INFO    :   EnergyReport - Energy [             a57]: 3.037161
12:38:57  INFO    : # Stop FTrace
12:38:59  INFO    : # Save FTrace: /home/derkling/Code/lisa/results/WlgenExample/example1/trace.dat
12:39:01  INFO    : # Save platform description: /home/derkling/Code/lisa/results/WlgenExample/example1/platform.json
12:39:01  INFO    : # Report collected data:
12:39:01  INFO    :    /home/derkling/Code/lisa/results/WlgenExample/example1


total 4008
drwxrwxr-x 2 derkling derkling    4096 Mar  2 12:39 .
drwxrwxr-x 3 derkling derkling    4096 Mar  2 12:38 ..
-rw-rw-r-- 1 derkling derkling      52 Mar  2 12:38 energy.json
-rw-r--r-- 1 derkling derkling     644 Mar  2 12:38 example1_00.json
-rw-rw-r-- 1 derkling derkling     235 Mar  2 12:38 output.log
-rw-rw-r-- 1 derkling derkling    1075 Mar  2 12:39 platform.json
-rw-r--r-- 1 derkling derkling    6360 Mar  2 12:38 rt-app-task_p20-0.log
-rw-r--r-- 1 derkling derkling    6360 Mar  2 12:38 rt-app-task_p20-3.log
-rw-r--r-- 1 derkling derkling 4063232 Mar  2 12:39 trace.dat


## 4) Check collected data

In [32]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

12:39:07  INFO    : Energy: /home/derkling/Code/lisa/results/WlgenExample/example1/energy.json


{
    "LITTLE": "5.125284", 
    "big": "3.037161"
}


In [33]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
print json.dumps(plt, indent=4, sort_keys=True)

12:39:07  INFO    : Platform description: /home/derkling/Code/lisa/results/WlgenExample/example1/platform.json


{
    "clusters": {
        "big": [
            1, 
            2
        ], 
        "little": [
            0, 
            3, 
            4, 
            5
        ]
    }, 
    "cpus_count": 6, 
    "freqs": {
        "big": [
            450000, 
            625000, 
            800000, 
            950000, 
            1100000
        ], 
        "little": [
            450000, 
            575000, 
            700000, 
            775000, 
            850000
        ]
    }, 
    "nrg_model": {
        "big": {
            "cluster": {
                "nrg_max": 64
            }, 
            "cpu": {
                "cap_max": 1024, 
                "nrg_max": 616
            }
        }, 
        "little": {
            "cluster": {
                "nrg_max": 57
            }, 
            "cpu": {
                "cap_max": 447, 
                "nrg_max": 93
            }
        }
    }, 
    "topology": [
        [
            0, 
            3, 
            4, 
        

## 5) trace inspection

In [34]:
!kernelshark {trace_file} 2>/dev/null

version = 6
trace-cmd: No such file or directory
  [vmscan:mm_vmscan_writepage] function sizeof not defined
  unknown op '~'
  Error: expected type 5 but read 0
  Error: expected type 4 but read 0
  unknown op '~'
  Error: expected type 5 but read 0
  Error: expected type 4 but read 0
  unknown op '~'
  Error: expected type 5 but read 0
  Error: expected type 4 but read 0
  unknown op '~'
  Error: expected type 5 but read 0
  Error: expected type 4 but read 0
  [sunrpc:xs_tcp_data_recv] function __builtin_constant_p not defined
  [sunrpc:xprt_transmit] function __builtin_constant_p not defined
  [sunrpc:xprt_lookup_rqst] function __builtin_constant_p not defined
  [sunrpc:xprt_complete_rqst] function __builtin_constant_p not defined
  [sunrpc:svc_send] function __builtin_constant_p not defined
  [sunrpc:svc_recv] function __builtin_constant_p not defined
  [sunrpc:svc_process] function __builtin_constant_p not defined
  [ras:mc_event] function mc_event_error_type not defined
  [libata:

# Workload composition using RTApp

## 1) creation

In [37]:
# Support to configure and run RTApp based workloads
from wlgen import RTA

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp_name = 'example2'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

12:43:40  INFO    :          WlGen - Setup new workload example2


## 2) configuration

In [40]:
# RTApp configurator for generation of PERIODIC tasks
from wlgen import Periodic, Ramp

cpus = str(target.bl.bigs_online[0])
# Light workload
light  = Periodic(duty_cycle_pct=10, duration_s=1.0, period_ms= 10,
              cpus=cpus)
# Ramp workload
ramp   = Ramp(start_pct=10, end_pct=90, delta_pct=20, time_s=1, period_ms=50,
             cpus=cpus)
# Heavy workload
heavy  = Periodic(duty_cycle_pct=90, duration_s=0.1, period_ms=100,
                 cpus=cpus)

# Composed workload
lrh_task = light + ramp + heavy

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_ramp': lrh_task.get() 

    },
    
);

12:44:39  INFO    :          RTApp - Workload duration defined by longest task
12:44:39  INFO    :          RTApp - Default policy: SCHED_OTHER
12:44:39  INFO    :          RTApp - ------------------------
12:44:39  INFO    :          RTApp - task [task_ramp], sched: using default policy
12:44:39  INFO    :          RTApp -  | calibration CPU: 1
12:44:39  INFO    :          RTApp -  | loops count: 1
12:44:39  INFO    :          RTApp -  | CPUs affinity: 1
12:44:39  INFO    :          RTApp - + phase_000001: duration 1.000000 [s] (100 loops)
12:44:39  INFO    :          RTApp - |  period    10000 [us], duty_cycle  10 %
12:44:39  INFO    :          RTApp - |  run_time   1000 [us], sleep_time   9000 [us]
12:44:39  INFO    :          RTApp - + phase_000002: duration 1.000000 [s] (20 loops)
12:44:39  INFO    :          RTApp - |  period    50000 [us], duty_cycle  10 %
12:44:39  INFO    :          RTApp - |  run_time   5000 [us], sleep_time  45000 [us]
12:44:39  INFO    :          RTApp - + 

In [41]:
# Inspect the JSON file used to run the application
with open('./{}_00.json'.format(rtapp_name), 'r') as fh:
    rtapp_json = json.load(fh)
logging.info('Generated RTApp JSON file:')
print json.dumps(rtapp_json, indent=4, sort_keys=True)

12:45:34  INFO    : Generated RTApp JSON file:


{
    "global": {
        "calibration": 138, 
        "default_policy": "SCHED_OTHER", 
        "duration": -1, 
        "logdir": "/root/devlib-target"
    }, 
    "tasks": {
        "task_ramp": {
            "cpus": [
                1
            ], 
            "loop": 1, 
            "phases": {
                "p000001": {
                    "loop": 100, 
                    "run": 1000, 
                    "timer": {
                        "period": 10000, 
                        "ref": "task_ramp"
                    }
                }, 
                "p000002": {
                    "loop": 20, 
                    "run": 5000, 
                    "timer": {
                        "period": 50000, 
                        "ref": "task_ramp"
                    }
                }, 
                "p000003": {
                    "loop": 20, 
                    "run": 15000, 
                    "timer": {
                        "period": 50000, 
                 

## 3) execution

In [42]:
res_dir = os.path.join(te.res_dir, rtapp_name)
nrg_report, plt, plt_file, trace_file = execute(te, rtapp, res_dir)

12:48:02  INFO    : # Create results folder for this execution
12:48:02  INFO    : # Setup FTrace
12:48:06  INFO    : ## Start energy sampling
12:48:07  INFO    : ### Start RTApp execution
12:48:07  INFO    :          WlGen - Workload execution START:
12:48:07  INFO    :          WlGen -    /root/devlib-target/bin/rt-app /root/devlib-target/example2_00.json
12:48:14  INFO    : ## Read energy consumption: /home/derkling/Code/lisa/results/WlgenExample/example2/energy.json
12:48:16  INFO    :   EnergyReport - Energy [             a53]: 1.495969
12:48:16  INFO    :   EnergyReport - Energy [             a57]: 25.145680
12:48:16  INFO    : # Stop FTrace
12:48:17  INFO    : # Save FTrace: /home/derkling/Code/lisa/results/WlgenExample/example2/trace.dat
12:48:19  INFO    : # Save platform description: /home/derkling/Code/lisa/results/WlgenExample/example2/platform.json
12:48:19  INFO    : # Report collected data:
12:48:19  INFO    :    /home/derkling/Code/lisa/results/WlgenExample/example2


total 4124
drwxrwxr-x 2 derkling derkling    4096 Mar  2 12:48 .
drwxrwxr-x 4 derkling derkling    4096 Mar  2 12:48 ..
-rw-rw-r-- 1 derkling derkling      53 Mar  2 12:48 energy.json
-rw-r--r-- 1 derkling derkling    2121 Mar  2 12:48 example2_00.json
-rw-rw-r-- 1 derkling derkling     235 Mar  2 12:48 output.log
-rw-rw-r-- 1 derkling derkling    1075 Mar  2 12:48 platform.json
-rw-r--r-- 1 derkling derkling   25084 Mar  2 12:48 rt-app-task_ramp-0.log
-rw-r--r-- 1 derkling derkling 4169728 Mar  2 12:48 trace.dat


## 4) trace inspection

In [48]:
!kernelshark {trace_file} 2>/dev/null

version = 6


# Custom RTApp connfiguration

In [51]:
# Support to configure and run RTApp based workloads
from wlgen import RTA

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp_name = 'example3'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp to use a custom JSON
rtapp.conf(

    # 1. generate a "custom" set of tasks
    kind = 'custom',
    
    # 2. define the "profile" of each task
    params = "../../assets/mp3-short.json",
    
    # In this case only few values of the orignal JSON can be tuned:
    #    DURATION : maximum duration of the workload [s]
    #    PVALUE   : calibration value
    #    LOGDIR   : folder used for generated logs
    #    WORKDIR  : working directory on target
    
    # 3. defined a maximum duration for that workload
    duration = 5,
    
);

12:55:36  INFO    :          WlGen - Setup new workload example3


In [52]:
res_dir = os.path.join(te.res_dir, rtapp_name)
nrg_report, plt, plt_file, trace_file = execute(te, rtapp, res_dir)

12:55:53  INFO    : # Create results folder for this execution
12:55:53  INFO    : # Setup FTrace
12:55:57  INFO    : ## Start energy sampling
12:55:58  INFO    : ### Start RTApp execution
12:55:58  INFO    :          WlGen - Workload execution START:
12:55:58  INFO    :          WlGen -    /root/devlib-target/bin/rt-app /root/devlib-target/example3_00.json
12:56:03  INFO    : ## Read energy consumption: /home/derkling/Code/lisa/results/WlgenExample/example3/energy.json
12:56:04  INFO    :   EnergyReport - Energy [             a53]: 2.481205
12:56:04  INFO    :   EnergyReport - Energy [             a57]: 8.981533
12:56:04  INFO    : # Stop FTrace
12:56:05  INFO    : # Save FTrace: /home/derkling/Code/lisa/results/WlgenExample/example3/trace.dat
12:56:07  INFO    : # Save platform description: /home/derkling/Code/lisa/results/WlgenExample/example3/platform.json
12:56:07  INFO    : # Report collected data:
12:56:07  INFO    :    /home/derkling/Code/lisa/results/WlgenExample/example3


total 5048
drwxrwxr-x 2 derkling derkling    4096 Mar  2 12:56 .
drwxrwxr-x 5 derkling derkling    4096 Mar  2 12:55 ..
-rw-rw-r-- 1 derkling derkling      52 Mar  2 12:56 energy.json
-rw-r--r-- 1 derkling derkling    1324 Mar  2 12:56 example3_00.json
-rw-rw-r-- 1 derkling derkling     807 Mar  2 12:56 output.log
-rw-rw-r-- 1 derkling derkling    1075 Mar  2 12:56 platform.json
-rw-r--r-- 1 derkling derkling 5144576 Mar  2 12:56 trace.dat


In [53]:
# Inspect the JSON file used to run the application
with open('./{}_00.json'.format(rtapp_name), 'r') as fh:
    rtapp_json = json.load(fh)
logging.info('Generated RTApp JSON file:')
print json.dumps(rtapp_json, indent=4, sort_keys=True)

12:56:21  INFO    : Generated RTApp JSON file:


{
    "global": {
        "calibration": 138, 
        "default_policy": "SCHED_OTHER", 
        "duration": 5, 
        "frag": 1, 
        "ftrace": false, 
        "gnuplot": false, 
        "lock_pages": true, 
        "log_basename": "mp3", 
        "logdir": "/root/devlib-target"
    }, 
    "tasks": {
        "AudioOut": {
            "loop": -1, 
            "priority": -19, 
            "resume": "AudioTrack", 
            "run": 4725, 
            "suspend": "AudioOut"
        }, 
        "AudioTick": {
            "cpus": [
                0
            ], 
            "loop": -1, 
            "phases": {
                "p1": {
                    "loop": 1, 
                    "resume": "AudioOut", 
                    "timer": {
                        "period": 6000, 
                        "ref": "tick"
                    }
                }, 
                "p2": {
                    "loop": 4, 
                    "timer": {
                        "period": 6000

# Running Hackbench

## 1) creation and configuration

In [72]:
# Support to configure and run RTApp based workloads
from wlgen import PerfMessaging

# Create a "perf bench sched messages" (i.e. hackbench) workload
perf_name = 'hackbench'
perf = PerfMessaging(target, perf_name)

perf.conf(group=1, loop=100, pipe=True, thread=True)


01:08:28  INFO    :          WlGen - Setup new workload hackbench


'hackbench_00'

## 2) execution

In [73]:
res_dir = os.path.join(te.res_dir, perf_name)
nrg_report, plt, plt_file, trace_file = execute(te, perf, res_dir)

01:08:29  INFO    : # Create results folder for this execution


mkdir: cannot create directory ‘/home/derkling/Code/lisa/results/WlgenExample/hackbench’: File exists


01:08:29  INFO    : # Setup FTrace
01:08:31  INFO    : ## Start energy sampling
01:08:31  INFO    : ### Start RTApp execution
01:08:31  INFO    :          WlGen - Workload execution START:
01:08:31  INFO    :          WlGen -    /root/devlib-target/bin/perf bench sched messaging --pipe --thread --group 1 --loop 100
01:08:32  INFO    :      PerfBench - Completion time: 0.126000, Performance 7.936508
01:08:32  INFO    : ## Read energy consumption: /home/derkling/Code/lisa/results/WlgenExample/hackbench/energy.json
01:08:33  INFO    :   EnergyReport - Energy [             a53]: 1.066525
01:08:33  INFO    :   EnergyReport - Energy [             a57]: 2.245705
01:08:33  INFO    : # Stop FTrace
01:08:34  INFO    : # Save FTrace: /home/derkling/Code/lisa/results/WlgenExample/hackbench/trace.dat
01:08:36  INFO    : # Save platform description: /home/derkling/Code/lisa/results/WlgenExample/hackbench/platform.json
01:08:36  INFO    : # Report collected data:
01:08:36  INFO    :    /home/derkling

total 10396
drwxrwxr-x 2 derkling derkling     4096 Mar  2 13:08 .
drwxrwxr-x 6 derkling derkling     4096 Mar  2 13:08 ..
-rw-rw-r-- 1 derkling derkling       52 Mar  2 13:08 energy.json
-rw-rw-r-- 1 derkling derkling      147 Mar  2 13:08 output.log
-rw-rw-r-- 1 derkling derkling       61 Mar  2 13:08 performance.json
-rw-rw-r-- 1 derkling derkling     1075 Mar  2 13:08 platform.json
-rw-r--r-- 1 derkling derkling 10620928 Mar  2 13:08 trace.dat


## 3) explore the performance report

In [76]:
# Inspect the generated performance report
perf_file = os.path.join(te.res_dir, perf_name, 'performance.json')
with open(perf_file, 'r') as fh:
    perf_json = json.load(fh)
logging.info('Generated performance JSON file:')
print json.dumps(perf_json, indent=4, sort_keys=True)

01:10:40  INFO    : Generated performance JSON file:


{
    "ctime": 0.126, 
    "performance": 7.936507936507937
}


## 4) trace inspection 

In [77]:
!kernelshark {trace_file} 2>/dev/null

version = 6
